Детектор аномалии на принципе многократной прогонки реконструкции входного образца до сходимости латентного вектора.
Критерий аномальности - расстояние от первоначального латентного вектора до сошедшего.

In [1]:
from dataclasses import dataclass
from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np 
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
from tqdm.auto import tqdm
from torchfuzzy import FuzzyLayer, DefuzzyLinearLayer, FuzzyBellLayer
import piqa
import sklearn.metrics as metrics
from sklearn.manifold import TSNE
from torchvision.transforms import v2

In [2]:
batch_size = 256
learning_rate = 2e-3
num_epochs = 300
latent_dim = 5
mnist_class_anomaly = 4
kernels = 2
fuzzy_rules_count = 100
beta = 1e-3
gamma = 1

prefix = f"fuzzy_cvae_mamdani_anomaly"
writer = SummaryWriter(f'runs/mnist/{prefix}_{datetime.now().strftime("%Y%m%d-%H%M%S")}')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ssim = piqa.SSIM(window_size = 11, n_channels=1, reduction='none').to(device)

## Датасет

1. Исключаем класс аномалии `mnist_class_anomaly` из общей выборк
2. Убираем метки с остальных классов
   

In [3]:
def norm_and_transform(x):
    nimg = 2.0*(x.view(-1, 28, 28) - 0.5)
    nimg = torch.clamp(nimg, -1, 1)
    return nimg

transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.RandomCrop(size=26),
    transforms.Resize(size=(28, 28)),
    transforms.Lambda(norm_and_transform)
])

In [ ]:
def get_target_and_mask(target_label):
    t = target_label
    return t 

train_data = datasets.MNIST(
    '~/.pytorch/MNIST_data/', 
    download=True, 
    train=True, 
    transform = transform,
    target_transform = transforms.Lambda(lambda x: get_target_and_mask(x))
)

idx = (train_data.targets != mnist_class_anomaly)
train_data.targets = train_data.targets[idx]
train_data.data = train_data.data[idx]
len(train_data)

загружаем тестовую выборку

In [ ]:
test_data = datasets.MNIST(
    '~/.pytorch/MNIST_data/', 
    download=True, 
    train=False, 
    transform=transform, 
    target_transform = transforms.Lambda(lambda x: get_target_and_mask(x))
)
len(test_data)

Создаем итераторы датасетов

In [6]:

train_loader = torch.utils.data.DataLoader(
    train_data, 
    batch_size=batch_size, 
    shuffle=True,
    
)
test_loader = torch.utils.data.DataLoader(
    test_data, 
    batch_size=batch_size, 
    shuffle=False,
)

In [ ]:
for data,_ in iter(train_loader):
    plt.imshow(data[0].squeeze())
    
    break

## Модель

In [8]:
class TermAverageActivationStatsLayer(torch.nn.Module):
    def __init__(self, terms_count, alpha=1e-2):
        """
        """
        super().__init__()
        #self.dummy = torch.nn.Parameter(torch.empty(0))
        self.terms_count = terms_count
        self.alpha = alpha
        self.accumulated_average_activation = None
        

    def forward(self, x):
        """
        """
        ava = x.mean(0)
        if self.accumulated_average_activation is None:
            self.accumulated_average_activation = ava
        else:
            self.accumulated_average_activation = (1 - self.alpha) * self.accumulated_average_activation + self.alpha * ava

        return x

    def get_norm_stats(self, eps=1e-7):
        """
        
        """
        s = self.accumulated_average_activation.sum() + eps
        return (self.accumulated_average_activation / s).detach()

In [9]:
class TermLatentSpaceCentroidTrackingLayer(torch.nn.Module):
    def __init__(self, latent_dim, alpha=1e-3):
        """
        """
        super().__init__()
        #self.dummy = torch.nn.Parameter(torch.empty(0))
        self.latent_dim = latent_dim
        self.alpha = alpha
        self.aver_centroid = None

    def forward(self, latent_vectors):
        """
        """
        avc = latent_vectors.mean(0)
        if self.aver_centroid is None:
            self.aver_centroid = avc
        else:
            self.aver_centroid = (1 - self.alpha) * self.aver_centroid + self.alpha * avc
        return latent_vectors

    def get_average_centroid(self):
        """
        
        """
        return self.aver_centroid.detach()

In [10]:
class Encoder(nn.Module):
    """
    Компонент энкодера для VAE
    
    Args:
        latent_dim (int): Размер латентного вектора.
    """
    
    def __init__(self, latent_dim):
        super(Encoder, self).__init__()
                
        self.input = nn.Sequential(
            nn.Conv2d(1, kernels, kernel_size=3, padding=2, stride=1),
            nn.BatchNorm2d(kernels, track_running_stats=False),
            nn.SiLU(),  
        )

        self.block_2 = nn.Sequential(
            nn.Conv2d(kernels, 2*kernels, kernel_size=2, stride=2, padding = 2),
            nn.BatchNorm2d(2*kernels, track_running_stats=False),
            nn.SiLU(),  
            nn.Conv2d(2*kernels, 2*kernels, kernel_size=3, stride=2, padding = 2),
            nn.BatchNorm2d(2*kernels, track_running_stats=False),
            nn.SiLU(),  
        )

        self.block_3 = nn.Sequential(
            nn.Conv2d(2*kernels, 4*kernels, kernel_size = 3, stride=1, padding = 0),
            nn.BatchNorm2d(4*kernels, track_running_stats=False),
            nn.SiLU(),  
            nn.Conv2d(4*kernels, 4*kernels, kernel_size = 3, stride=1, padding = 0),
            nn.BatchNorm2d(4*kernels, track_running_stats=False),
            nn.SiLU(),  
        )

        self.block_4 = nn.Sequential(
            nn.Conv2d(4*kernels, 8*kernels, kernel_size = 4, stride=1, padding = 0),
            nn.BatchNorm2d(8*kernels, track_running_stats=False),
            nn.SiLU(),  
            nn.Conv2d(8*kernels, 8*kernels, kernel_size = 3, stride=1, padding = 0),
            nn.BatchNorm2d(8*kernels, track_running_stats=False),
            nn.SiLU(),  
        )

        self.out = nn.Sequential(
            nn.Flatten(),
            nn.Linear(8*kernels, 2 * latent_dim), # mean + variance.
        )

        self.downscale_1 = nn.Conv2d(kernels, 8*kernels, kernel_size=30)
        self.downscale_2 = nn.Conv2d(2*kernels, 8*kernels, kernel_size=10)
        self.downscale_3 = nn.Conv2d(4*kernels, 8*kernels, kernel_size=6)
        self.after_sum = nn.SiLU()
        self.softplus = nn.Softplus()

         
    def forward(self, x, eps: float = 1e-8):
        """
        Выход энкодера для чистого VAE.
        
        Args:
            x (torch.Tensor): Входной вектор.
            eps (float): Небольшая поправка к скейлу для лучшей сходимости и устойчивости.
        
        Returns:
            mu, logvar, z, dist
        """

        x = self.input(x)
        #print(x.shape)
        res_1 = self.downscale_1(x)
        

        x = self.block_2(x)
        #print(x.shape)
        res_2 = self.downscale_2(x)
        
        x = self.block_3(x)
        #print(x.shape)
        res_3 = self.downscale_3(x)
        
        x = self.block_4(x)

        x = self.after_sum(x + res_1 + res_2 + res_3)
        x = self.out(x)

        mu, logvar = torch.chunk(x, 2, dim=-1)
        scale = self.softplus(logvar) + eps
        scale_tril = torch.diag_embed(scale)
        dist = torch.distributions.MultivariateNormal(mu, scale_tril=scale_tril)
        z = dist.rsample()
        
        return mu, logvar, z

In [ ]:
inp = torch.rand(10, 1, 28, 28)
m = Encoder(latent_dim)
mu = m.forward(inp)
mu[0].shape

In [12]:
class Decoder(nn.Module):
    """
    Компонент декодера для VAE
    
    Args:
        latent_dim (int): Размер латентного вектора.
    """
    
    def __init__(self, latent_dim, fuzzy_rules_count):
        super(Decoder, self).__init__()

        initial_centroids = 2 * (0.5 - np.random.rand(fuzzy_rules_count, latent_dim))
        initial_scales = np.ones((fuzzy_rules_count, latent_dim))       
        self.fuzzy = nn.Sequential(
            TermLatentSpaceCentroidTrackingLayer(latent_dim),
            FuzzyLayer.from_centers_and_scales(initial_centroids, initial_scales, trainable=True),
            TermAverageActivationStatsLayer(fuzzy_rules_count)
        )

        self.decoder = nn.Sequential(
            nn.Linear(fuzzy_rules_count, 16*kernels),
            nn.SiLU(), 
            nn.BatchNorm1d(16*kernels, track_running_stats=False),
            nn.Unflatten(1, (16*kernels, 1, 1)),
            nn.ConvTranspose2d(16*kernels, 8*kernels, 12),
            nn.SiLU(), 
            nn.BatchNorm2d(8*kernels, track_running_stats=False),
            nn.ConvTranspose2d(8*kernels, 4*kernels, 5),
            nn.SiLU(), 
            nn.BatchNorm2d(4*kernels, track_running_stats=False),
            nn.ConvTranspose2d(4*kernels, 2*kernels, 5),
            nn.SiLU(), 
            nn.BatchNorm2d(2*kernels, track_running_stats=False),
            nn.ConvTranspose2d(2*kernels, kernels, 5),
            nn.SiLU(), 
            nn.BatchNorm2d(kernels, track_running_stats=False),
            nn.ConvTranspose2d(kernels, 1, 5),
            nn.Tanh() 
        )
         
    def forward(self, z):
        """
        Декодирует латентный вектор в исходное представление
        
        Args:
            z (torch.Tensor): Латентный вектор.
        
        Returns:
            x
        """
        fz = self.fuzzy(z)
        x = self.decoder(fz)
        return x, fz

In [ ]:
inp = torch.rand(10, latent_dim)
m = Decoder(latent_dim, fuzzy_rules_count=fuzzy_rules_count)
mu, fz = m.forward(inp)
mu.shape

In [14]:
class VAE(nn.Module):
    """
    Args:
        latent_dim (int): Размер латентного вектора.
    """
    def __init__(self, latent_dim, fuzzy_rules_count):
        super(VAE, self).__init__()

        self.encoder = Encoder(latent_dim)        
        self.decoder = Decoder(latent_dim, fuzzy_rules_count)
        

    def forward(self, x):
        """
        
        """
        mu, _,  _, = self.encoder(x)
        x_recon, fz = self.decoder(mu)
        return mu, x_recon, fz
    
    def half_pass(self, x):
        """
        
        """
        mu, logvar, z = self.encoder(x)
        return mu, logvar, z
    
    def decoder_pass(self, x):
        return self.decoder(x)
    
    def fuzzy_pass(self, x):
        return self.decoder.fuzzy(x)


## Обучение

In [ ]:
model = VAE(latent_dim=latent_dim, fuzzy_rules_count=fuzzy_rules_count).to(device)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of parameters: {num_params:,}')

model

In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer_d = torch.optim.Adam(model.decoder.fuzzy[1].parameters(), lr=learning_rate)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

In [ ]:
fz = torch.rand(batch_size, latent_dim)
max_entropy = np.log(1.0/latent_dim)
p = nn.functional.normalize(fz.sum(0), p=1, dim=0) + 1e-7
(p*p.log()).sum() - max_entropy

In [18]:
def compute_vae_loss(x, recon_x, mu, logvar):
    
    diff = ssim((x + 1)/2, (recon_x+1)/2)
    
    loss_recon = (1 - diff).abs().mean() #F.binary_cross_entropy((recon_x+1)/2, (x + 1)/2, reduction='none').sum(-1).mean()#
    
    tsquare = torch.square(mu)
    tlogvar = torch.exp(logvar)
    kl_loss = -0.5 * (1 + logvar - tsquare - tlogvar)
    loss_kl = kl_loss.sum(-1).mean()
    
    loss = loss_recon + beta * loss_kl

    return loss, loss_recon, loss_kl
    

In [19]:
# inp = torch.rand(10, latent_dim).to(device)
# eigensum = (100 - fuzzy_layer.get_transformation_matrix_eigenvals().real.mean(-1)).square()
# (model.decoder_pass(inp)[1]* eigensum).sum(-1).mean()

In [20]:
def compute_centroids_loss(fuzzy_layer):
    
    c = fuzzy_layer[1].get_centroids()
    c = c.mean(0)
    ac = fuzzy_layer[0].get_average_centroid()

    return (c-ac).square().mean()


In [21]:
# fz_inference = torch.rand(3, fuzzy_rules_count)
# winners = fz_inference.argmax(-1)
eigens = model.decoder.fuzzy[1].get_transformation_matrix_eigenvals().real


# eigens[winners]

In [22]:
def compute_fuzzy_loss(fz_inference, fuzzy_layer):
    
    fz_loss = (0.8 - torch.topk(fz_inference, 3).values.sum(-1).clamp(max=0.8)).square().mean() #fz_inference.min(-1).values.square().mean()  #(1 - fz.sum(-1)).square() + (1 - fz.max(-1).values).square() .mean() #fz.quantile(0.5, dim=-1).square().mean() # + (fz.min(-1).values).square().mean()
    
    eigens = fuzzy_layer[1].get_transformation_matrix_eigenvals().real

    #winners = fz_inference.argmax(-1)
    fz_volume = (0.8 - (eigens.min(-1).values/eigens.max(-1).values).clamp(max=0.8)).square()
    fz_volume = fz_volume.mean()
    return fz_loss, fz_volume
    

In [23]:
def keep_eigenvals_positive_loss(layer, eps = 1e-15):
    ev = layer[1].get_transformation_matrix_eigenvals().real.min()
    ev = torch.clamp(ev, max=eps)
    return -ev

In [24]:
def get_arate(inp):
    _, _, fz = model.forward(inp)
    # prev_mu = mu
    # sum = torch.zeros_like(mu)
    
    # for i in range(20):
    #     recon_x = model.decoder_pass(mu)
    #     mu, _, _ = model.half_pass(recon_x)
    #     sum += (mu - prev_mu).abs()
    #     prev_mu = mu
    # +
    return fz.sum(-1).cpu().numpy()#ssim((inp + 1)/2, (recon_x+1)/2).cpu().numpy() #

In [25]:
def train(model, dataloader, optimizer_e, prev_updates, writer=None):
    model.train()  
    
    for batch_idx, (data, _) in enumerate(tqdm(dataloader, disable=True)):
        n_upd = prev_updates + batch_idx
        
        data = data.to(device)
        
        optimizer_e.zero_grad()  
        
        mu, logvar, z = model.half_pass(data)  
        recon_x, fz = model.decoder_pass(z)
        loss, _, _ = compute_vae_loss(data, recon_x, mu, logvar)
        fz_loss, fz_volume = compute_fuzzy_loss(fz, model.decoder.fuzzy)                
        ev_loss = keep_eigenvals_positive_loss(model.decoder.fuzzy)
        #c_loss = compute_centroids_loss(model.decoder.fuzzy)

        if ev_loss.item() > 0:
            ev_loss.backward(retain_graph=True)
        fz_loss.backward(retain_graph=True)
        fz_volume.backward(retain_graph=True)
        #c_loss.backward(retain_graph=True)
        loss.backward()

        optimizer_e.step()  

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)    
        
        
    return prev_updates + len(dataloader)

In [26]:
def shannon_entropy(fuzzy):
    distr = fuzzy[2].get_norm_stats()
    dim = distr.shape[0]
    return np.log(dim) + (distr*distr.log()).sum()

In [27]:
fixed_random_z = torch.randn(16, latent_dim).to(device)

def test(model, dataloader, cur_step, epoch, writer=None):
    model.eval() 
    test_recon_loss = 0
    test_kl_loss = 0
    test_fz_loss = 0
    test_fzvol_loss = 0
    test_c_loss = 0
    
    lab_true = []
    lab_pred = []

    with torch.no_grad():
        for data, lab in tqdm(test_loader, desc='Test MNIST', disable=True):
            data = data.view((-1,1,28,28)).to(device)
            rates = get_arate(data)
            
            for f, l in  zip(rates, lab):
                lab_pred.append(f)        
                if l == mnist_class_anomaly:
                    lab_true.append(1)
                else:
                    lab_true.append(0)
                        
    fpr, tpr, _ = metrics.roc_curve(lab_true, lab_pred)
    roc_auc = metrics.auc(fpr, tpr)
    
    embedings = []
    labels_expected = []
    centroids = model.decoder.fuzzy[1].get_centroids().detach().cpu().numpy()
    
    with torch.no_grad():
        for data, target in tqdm(dataloader, desc='Testing', disable=True):
            data = data.to(device)
            mu, logvar, z = model.half_pass(data)  
            recon_x, fz = model.decoder_pass(z)
            
            embedings.append(mu.cpu().numpy())
            labels_expected.append(target.cpu().numpy())

            _, loss_recon, loss_kl = compute_vae_loss(data, recon_x, mu, logvar)
            fz_loss, fz_volume = compute_fuzzy_loss(fz, model.decoder.fuzzy)    
            c_loss = compute_centroids_loss(model.decoder.fuzzy)    
                    
            test_recon_loss += loss_recon.item()
            test_kl_loss += loss_kl.item()
            test_fz_loss += fz_loss.item()
            test_fzvol_loss += fz_volume.item()
            test_c_loss += c_loss.item()

    embedings = np.concatenate(embedings, axis=0)
    labels_expected = np.concatenate(labels_expected, axis=0)

    if epoch % 20 == 1:
        plt.figure(figsize=(18, 6))

        R, C = 1, 3

        plt.subplot(R, C, 1)
        plt.title("MNIST XY")
        plt.scatter(embedings[:, 0],      embedings[:,  1], c=labels_expected, cmap='tab10', s=2)
        plt.scatter(centroids[:, 0],      centroids[:, 1], marker='1', c='black', s= 50)
        # plt.xlim((-1, 1))
        # plt.ylim((-1, 1))

        plt.subplot(R, C, 2)
        plt.title("MNIST XZ")
        plt.scatter(embedings[:, 0],      embedings[:,  2], c=labels_expected, cmap='tab10', s=2)
        plt.scatter(centroids[:, 0],      centroids[:, 2], marker='1', c='black', s= 50)
        # plt.xlim((-1, 1))
        # plt.ylim((-1, 1))

        plt.show()
    
    test_recon_loss /= len(dataloader)
    test_kl_loss /= len(dataloader)
    test_fz_loss /= len(dataloader)
    test_fzvol_loss /= len(dataloader)
    test_c_loss /= len(dataloader)
    print(f'[{cur_step}] Reconstruction loss: {test_recon_loss:.4f}, KLD: {test_kl_loss:.4f} AUC {roc_auc:.4f} FZ {test_fz_loss:.4f} FZVOL {test_fzvol_loss:.4f} CENTROIDS DISCREPANCY {test_c_loss:.4f} SHANNON {shannon_entropy(model.decoder.fuzzy)}')
    #print(f'Average activation stats: {model.decoder.fuzzy[2].get_norm_stats()}')
    #print(f'Average centroid stats: {model.decoder.fuzzy[0].get_average_centroid()}')
    if writer is not None:
        
        writer.add_scalar('ADFVAE/AUC', roc_auc, global_step=cur_step)
        writer.add_scalar('ADFVAE/Reconstruction', test_recon_loss, global_step=cur_step)
        writer.add_scalar('ADFVAE/KLD', test_kl_loss, global_step=cur_step)
        writer.add_scalar('ADFVAE/Fuzzy', test_fz_loss, global_step=cur_step)
        writer.add_scalar('ADFVAE/Fuzzy/Vol', test_fzvol_loss, global_step=cur_step)
        writer.add_scalar('ADFVAE/Fuzzy/CDist', test_c_loss, global_step=cur_step)
        
        samples, _ = model.decoder_pass(fixed_random_z)
        writer.add_images('ADFVAE/Samples', samples.view(-1, 1, 28, 28), global_step=cur_step)

In [28]:
prev_updates = 0

In [ ]:
for epoch in range(num_epochs):
    prev_updates = train(model, train_loader, optimizer, prev_updates, writer=writer)
    test(model, test_loader, prev_updates, epoch, writer=writer)
    scheduler.step()


In [ ]:
keep_eigenvals_positive_loss(model.decoder.fuzzy)

## Визуализируем результаты

In [ ]:

def get_activation_stats(model, dataloader):
    rulestat = {}
    with torch.no_grad():
        for _, (data, _) in enumerate(tqdm(dataloader)):
            data = data.to(device)
            _, _, fz = model.forward(data)
            act_fz = fz.max(-1).indices.cpu().numpy()
            for ind in act_fz:
                rulestat[ind] = rulestat.get(ind, 0) + 1
    return rulestat

train_stat = get_activation_stats(model, train_loader)
test_stat = get_activation_stats(model, test_loader)
plt.figure(figsize=(18, 6))
plt.bar(list(train_stat.keys()), train_stat.values(), 0.5, color='g')
plt.bar(list(test_stat.keys()), test_stat.values(), 0.5, color='r')

In [ ]:
train_stat

In [ ]:
def get_activation_stats_by_digit(digit, model, dataloader):
    rulestat = {}
    with torch.no_grad():
        for _, (data, target) in enumerate(tqdm(dataloader)):
            data = data.to(device)
            _, _, fz = model.forward(data)
            act_fz = fz.max(-1).indices.cpu().numpy()
            for ind, lab in zip(act_fz, target.cpu().numpy()):
                if lab == digit:
                    rulestat[ind] = rulestat.get(ind, 0) + 1
    return rulestat


test_stat_by_digit_0 = get_activation_stats_by_digit(0, model, test_loader)
test_stat_by_digit_1 = get_activation_stats_by_digit(1, model, test_loader)
test_stat_by_digit_2 = get_activation_stats_by_digit(2, model, test_loader)
test_stat_by_digit_3 = get_activation_stats_by_digit(3, model, test_loader)
test_stat_by_digit_4 = get_activation_stats_by_digit(4, model, test_loader)
test_stat_by_digit_5 = get_activation_stats_by_digit(5, model, test_loader)
test_stat_by_digit_6 = get_activation_stats_by_digit(6, model, test_loader)
test_stat_by_digit_7 = get_activation_stats_by_digit(7, model, test_loader)
test_stat_by_digit_8 = get_activation_stats_by_digit(8, model, test_loader)
test_stat_by_digit_9 = get_activation_stats_by_digit(9, model, test_loader)


plt.figure(figsize=(18, 6))
plt.bar(list(test_stat_by_digit_0.keys()), test_stat_by_digit_0.values(), 1)
plt.bar(list(test_stat_by_digit_1.keys()), test_stat_by_digit_1.values(), 1)
plt.bar(list(test_stat_by_digit_2.keys()), test_stat_by_digit_2.values(), 1)
plt.bar(list(test_stat_by_digit_3.keys()), test_stat_by_digit_3.values(), 1)
plt.bar(list(test_stat_by_digit_4.keys()), test_stat_by_digit_4.values(), 1)
plt.bar(list(test_stat_by_digit_5.keys()), test_stat_by_digit_5.values(), 1)
plt.bar(list(test_stat_by_digit_6.keys()), test_stat_by_digit_6.values(), 1)
plt.bar(list(test_stat_by_digit_7.keys()), test_stat_by_digit_7.values(), 1)
plt.bar(list(test_stat_by_digit_8.keys()), test_stat_by_digit_8.values(), 1)
plt.bar(list(test_stat_by_digit_9.keys()), test_stat_by_digit_9.values(), 1)

In [ ]:
def get_fz_activations(model, dataloader):
    embedding = []
    labels = []
    with torch.no_grad():
        for _, (data, target) in enumerate(tqdm(dataloader)):
            data = data.to(device)
            _, _, fz = model.forward(data)
            fz_inf = fz.cpu().numpy()
            for f,l in zip(fz_inf, target.cpu().numpy()):
                embedding.append(f)
                labels.append(l)
                
    return embedding, labels

train_fz = get_fz_activations(model, train_loader)
test_fz = get_fz_activations(model, test_loader)

In [ ]:
from sklearn.ensemble import IsolationForest
X = train_fz[0]

clf = IsolationForest(random_state=0,n_estimators=1000).fit(X)

clf.predict(test_fz[0]).min()

In [ ]:
z = torch.randn(64, latent_dim).to(device)
samples,_ = model.decoder_pass(z)

# Plot the generated images
fig, ax = plt.subplots(8, 8, figsize=(8, 8))
for i in range(8):
    for j in range(8):
        ax[i, j].imshow(samples[i*8+j].view(28, 28).cpu().detach().numpy(), cmap='gray')
        ax[i, j].axis('off')

# plt.show()
plt.savefig('cvae_mnist.webp')

In [34]:
def get_arate(inp):
    _, _, fz = model.forward(inp)
    # +
    return (1-fz.topk(k=1).values.min(-1).values).cpu().numpy() #(fz.quantile(0.9, dim=-1)).cpu().numpy()#ssim((inp + 1)/2, (recon_x+1)/2).cpu().numpy() #

In [ ]:
firings_mnist = {}
firings_mnist['MNIST'] = []
firings_mnist['DISSIDENT'] = []

with torch.no_grad():
    for data, target in tqdm(test_loader, desc='MNIST HIST'):
        data = data.view((-1,1,28,28)).to(device)
        rates = get_arate(data)
        for f, l in  zip(rates, target):
            if l != mnist_class_anomaly:
                firings_mnist['MNIST'].append(f)
            else:
                firings_mnist['DISSIDENT'].append(f)
        

labels, data = firings_mnist.keys(), firings_mnist.values()

fig = plt.figure(figsize =(12, 2))
plt.boxplot(data, notch=True, showfliers=False)
plt.xticks(range(1, len(labels) + 1), labels)
plt.show()

writer.add_figure('Anomaly Detection', fig)

In [ ]:
with torch.no_grad():
    firing_levels = []
    lab_true = []
    lab_pred = []

    for data, lab in tqdm(test_loader, desc='Test MNIST', disable=True):
        data = data.view((-1,1,28,28)).to(device)
        rates = get_arate(data)
        
        for f, l in  zip(rates, lab):
            firing_levels.append(f)
            lab_pred.append(f)        
            if l == mnist_class_anomaly:
                lab_true.append(1)
            else:
                lab_true.append(0)
                    
    fpr, tpr, threshold = metrics.roc_curve(lab_true, lab_pred)
    roc_auc = metrics.auc(fpr, tpr)
    
    fig = plt.figure(figsize =(4, 4))
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    writer.add_figure('ROC', fig)

In [ ]:
def show_plot():
    centroids = model.decoder.fuzzy[1].get_centroids().detach().cpu().numpy()
    embedings = []
    labels_expected = []
    with torch.no_grad():
        for data, target in tqdm(test_loader, desc='Encoding'):
            data = data.view((-1,1,28,28)).to(device)
            embeding,_,_ = model.forward(data)
            embedings.append(embeding.cpu().numpy())
            labels_expected.append(target.cpu().numpy())
    embedings = np.concatenate(embedings, axis=0)
    labels_expected = np.concatenate(labels_expected, axis=0)

    plt.figure(figsize=(18, 6))

    R, C = 1, 3

    plt.subplot(R, C, 1)
    plt.title("MNIST XY")
    plt.scatter(embedings[:, 0],      embedings[:,  1], c=labels_expected, cmap='tab10', s=2)
    plt.scatter(centroids[:, 0],      centroids[:, 1], marker='1', c='black', s= 50)

    plt.subplot(R, C, 2)
    plt.title("MNIST XZ")
    plt.scatter(embedings[:, 0],      embedings[:,  2], c=labels_expected, cmap='tab10', s=2)
    plt.scatter(centroids[:, 0],      centroids[:, 2], marker='1', c='black', s= 50)

    
show_plot()